Importing Dependencies


Defining Parameters

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
from tensorflow.keras.constraints import MinMaxNorm
import numpy as np
from numpy import random

from PIL import Image
from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt
from tensorflow.keras.mixed_precision import experimental as mixed_precision

#format time into something more readable
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

import torch



Check for GPU

In [4]:
# tf.config.list_physical_devices('GPU')
# tf.test.gpu_device_name()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2070'

In [3]:
generate_res = (64,64)
PHOTO_PATH = "./data"
DATA_PATH = "training.npy"
MODEL_PATH = "./newmodels"
SAVE_PATH = './newmodels'
SEED_SIZE = 128
BATCH_SIZE = 128
EPOCHS = 750      ## Increase this for better generated image 
img_width = 64
img_height = 64
channels = 3
n_critic = 5
learning_rate = 1e-4
beta1 = 0
beta2 = 0.9

First we'll need to convert the images to numpy arrays and store as training.npy

In [4]:
training_data = []
for filename in tqdm(os.listdir(PHOTO_PATH)):
    path = os.path.join(PHOTO_PATH,filename)
    image = Image.open(path).resize((img_width,
            img_height),Image.ANTIALIAS)
    training_data.append(np.asarray(image))
training_data = np.reshape(training_data,(-1,img_width,
            img_height,channels))
training_data = training_data.astype(np.float32)
training_data = training_data / 127.5 - 1 #images should be normalised to [-1,1]
print(np.shape(training_data))
np.save("training.npy",training_data) 

100%|██████████| 21551/21551 [00:18<00:00, 1194.24it/s]


(21551, 64, 64, 3)


Define Generator Model

In [1]:
# Aryan

def build_generator(seed):
    model = Sequential()
    model.add(Conv2DTranspose(512, input_shape = (1,1,seed), kernel_size=4, strides=1, padding='valid',use_bias=False))
    model.add(Activation("relu"))
    
    #model.add(Dropout(0.5))
    model.add(Conv2DTranspose(256, kernel_size=4, strides=2, padding='same',use_bias=False))
    #model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    #model.add(Dropout(0.5))
    model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding='same',use_bias=False))
    #model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    #model.add(Dropout(0.5))
    model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding='same',use_bias=False))
    #model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    #model.add(Dropout(0.5))
    model.add(Conv2DTranspose(3, kernel_size=4, strides=2, padding='same',use_bias=False))
    model.add(Activation("tanh", dtype = 'float32'))
    model.summary()
    return model


Define Descriminator model

In [2]:
#Aryan

def build_discriminator(input_size):
    
    model = Sequential()

    model.add(Conv2D(64, kernel_size=4, strides=2, input_shape=input_size, padding="same"))
    #model.add(GaussianNoise(1))
    model.add(LeakyReLU(alpha=0.2))
    
    #model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=4, strides=2, padding="same",use_bias=False))
    #model.add(GaussianNoise(1))
    #model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    
    #model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=4, strides=2, padding="same",use_bias=False))
    #model.add(GaussianNoise(1))
    #model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    
    #model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=4, strides=2, padding="same",use_bias=False))
    #model.add(GaussianNoise(1))
    #model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    #model.add(Dropout(0.25))
    model.add(Conv2D(1, kernel_size=4, strides=1, padding="valid",use_bias=False))
    model.add(Flatten())
    model.add(Activation('linear', dtype = 'float32'))
    model.summary()
    return model



Loading saved images such that training_data % batch_size == 0


In [7]:
training_data = np.load(DATA_PATH)
train_dataset = tf.data.Dataset.from_tensor_slices(training_data[:int(len(training_data)/BATCH_SIZE)*BATCH_SIZE]).shuffle(int(len(training_data)/BATCH_SIZE)*BATCH_SIZE).batch(BATCH_SIZE)
training_data = []

Initializing generator and discriminator model

In [8]:
discriminator = build_discriminator((img_height,img_width,channels))#tf.keras.models.load_model("/kaggle/working/face_discriminator.h5")#
generator = build_generator(SEED_SIZE)#tf.keras.models.load_model("/kaggle/working/face_generator.h5")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        3136      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       131072    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 256)         524288    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 512)         2

Define gen and disc loss as well as Adam optimizers

In [9]:
def discriminator_loss(real_output,fake_output,penalty):
    return tf.reduce_mean(fake_output) - tf.reduce_mean(real_output) + penalty

def generator_loss(seed):
    fake_images = generator(seed,training=True)
    fake_output = discriminator(fake_images,training=True)
    return -tf.reduce_mean(fake_output)

generator_optimizer = Adam(learning_rate,beta1,beta2) 
#generator_optimizer = mixed_precision.LossScaleOptimizer(generator_optimizer, loss_scale='dynamic')

discriminator_optimizer = Adam(learning_rate,beta1,beta2)
#discriminator_optimizer = mixed_precision.LossScaleOptimizer(discriminator_optimizer, loss_scale='dynamic')

Discriminator train dunction

In [10]:
@tf.function
def train_disc(real_images,seed):
    with tf.GradientTape() as disc_tape:
        
        fake_images = generator(seed,training=True)
        real_output = discriminator(real_images,training=True)
        fake_output = discriminator(fake_images,training=True)
        g_penalty = gradient_penalty(real_images,fake_images)
        disc_loss = discriminator_loss(real_output,fake_output,g_penalty)
        #scaled_disc_loss = discriminator_optimizer.get_scaled_loss(disc_loss)
        
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)#(scaled_disc_loss, discriminator.trainable_variables)
        #gradients_of_discriminator = discriminator_optimizer.get_unscaled_gradients(gradients_of_discriminator)
    
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return tf.reduce_mean(real_output),g_penalty

Function to calculate Gradient Penalty 

In [11]:
def gradient_penalty(real_images,fake_images):
    t = np.random.uniform(size=[BATCH_SIZE, 1, 1, 1], low=0., high=1.).astype("float32")
    penalty_images = t* fake_images + (1-t)* real_images
    penalty_output = discriminator(penalty_images,training=True)
    penalty_grads = tf.gradients(penalty_output, [penalty_images])[0]
    slopes = tf.sqrt(1e-8 + tf.reduce_sum(tf.square(penalty_grads), axis=[1, 2, 3]))
    gradient_penalty = tf.reduce_mean((slopes - 1.) ** 2)            
    gradient_penalty = gradient_penalty * 10
    
    return gradient_penalty


Generator train function


In [12]:
@tf.function
def train_gen(seed):
    with tf.GradientTape() as gen_tape:
        gen_loss = generator_loss(seed)
        
        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)#(scaled_gen_loss, generator.trainable_variables)
        #gradients_of_generator = generator_optimizer.get_unscaled_gradients(gradients_of_generator) 
        
        #scaled_gen_loss = generator_optimizer.get_scaled_loss(gen_loss)
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    
    return tf.reduce_mean(gen_loss)

Training Loop


In [13]:
#Harsh


def train(dataset, epochs):
  start = time.time()
  for epoch in range(epochs):
    epoch_start = time.time()

    gen_loss_list = []
    disc_loss_list = []
    penalty_list = []

    for i,image_batch in enumerate(dataset):
      seed = np.random.normal(size=[BATCH_SIZE,1,1, SEED_SIZE]).astype("float32")
      disc_loss, penalty = train_disc(image_batch,seed)
      disc_loss_list.append(disc_loss)
      penalty_list.append(penalty)

      
      if i % n_critic == 0:
        gen_loss = train_gen(seed)
        gen_loss_list.append(gen_loss)
       
      
    g_loss = sum(gen_loss_list) / len(gen_loss_list)
    d_loss = sum(disc_loss_list) / len(disc_loss_list)
    penalty = sum(penalty_list) / len(penalty_list)
    
    if epoch%20 == 0:
        
        generator.save(os.path.join(SAVE_PATH,"face_generator.h5"))
        discriminator.save(os.path.join(SAVE_PATH,"face_discriminator.h5"))

    epoch_elapsed = time.time()-epoch_start
    print (f'Epoch {epoch+1}, fake output={g_loss},real output={d_loss}, penalty = {penalty}, {epoch_elapsed}')
        

  elapsed = time.time()-start
  print (f'Training time: {(elapsed)}')

Run training save model

In [14]:
train(train_dataset,EPOCHS)
generator.save(os.path.join(SAVE_PATH,"face_generator.h5"))
discriminator.save(os.path.join(SAVE_PATH,"face_discriminator.h5"))
print(SEED_SIZE)

Epoch 1, fake output=-8.490306854248047,real output=180.7479248046875, penalty = 60.237457275390625, 34.23403215408325
Epoch 2, fake output=-1.3771722316741943,real output=200.8270721435547, penalty = 73.65443420410156, 25.717946529388428
Epoch 3, fake output=-5.859780788421631,real output=213.4683380126953, penalty = 77.19000244140625, 25.933629751205444
Epoch 4, fake output=-26.25288963317871,real output=235.25656127929688, penalty = 77.65174865722656, 25.405811071395874
Epoch 5, fake output=-48.21764373779297,real output=250.07391357421875, penalty = 74.19554901123047, 25.341951847076416
Epoch 6, fake output=-73.8658676147461,real output=253.7664337158203, penalty = 64.57659149169922, 25.953073501586914
Epoch 7, fake output=-82.3429183959961,real output=227.2683868408203, penalty = 49.35785675048828, 25.356806755065918
Epoch 8, fake output=-67.20259857177734,real output=193.09078979492188, penalty = 40.899349212646484, 25.27056884765625
Epoch 9, fake output=-54.11255645751953,real o

Epoch 61, fake output=7.436233043670654,real output=40.13119125366211, penalty = 12.022786140441895, 27.396792888641357
Epoch 62, fake output=7.054713726043701,real output=39.33530044555664, penalty = 11.783089637756348, 26.736095905303955
Epoch 63, fake output=6.196065425872803,real output=38.30731964111328, penalty = 11.618809700012207, 27.246073246002197
Epoch 64, fake output=7.091370582580566,real output=39.06886672973633, penalty = 11.464909553527832, 28.575098276138306
Epoch 65, fake output=6.291149139404297,real output=39.377044677734375, penalty = 11.233451843261719, 28.068796157836914
Epoch 66, fake output=5.9812750816345215,real output=38.117584228515625, penalty = 11.027070999145508, 27.42194962501526
Epoch 67, fake output=6.7262725830078125,real output=37.23734664916992, penalty = 10.914318084716797, 27.727887868881226
Epoch 68, fake output=6.991800785064697,real output=37.24158477783203, penalty = 10.833324432373047, 27.844903230667114
Epoch 69, fake output=6.9917292594909

Epoch 121, fake output=11.647773742675781,real output=16.467567443847656, penalty = 5.906982421875, 26.638927698135376
Epoch 122, fake output=12.184998512268066,real output=15.48481273651123, penalty = 5.827762126922607, 26.915998458862305
Epoch 123, fake output=12.401012420654297,real output=14.805258750915527, penalty = 5.780777931213379, 27.034001111984253
Epoch 124, fake output=12.115614891052246,real output=15.208531379699707, penalty = 5.759244441986084, 27.339001655578613
Epoch 125, fake output=12.57346248626709,real output=14.632387161254883, penalty = 5.693770408630371, 27.014000415802002
Epoch 126, fake output=13.005531311035156,real output=14.244207382202148, penalty = 5.62783145904541, 27.097999572753906
Epoch 127, fake output=13.190679550170898,real output=14.311439514160156, penalty = 5.645094871520996, 27.350793600082397
Epoch 128, fake output=13.581117630004883,real output=13.40869140625, penalty = 5.576987266540527, 27.980096340179443
Epoch 129, fake output=14.00434494

Epoch 181, fake output=9.192448616027832,real output=12.380131721496582, penalty = 4.130669593811035, 26.839999437332153
Epoch 182, fake output=9.116602897644043,real output=13.219807624816895, penalty = 4.150962829589844, 26.748996257781982
Epoch 183, fake output=9.243864059448242,real output=12.922538757324219, penalty = 4.1230902671813965, 26.57647728919983
Epoch 184, fake output=9.124666213989258,real output=12.787571907043457, penalty = 4.0868706703186035, 26.857028484344482
Epoch 185, fake output=8.935737609863281,real output=12.765172958374023, penalty = 4.104735374450684, 26.79900026321411
Epoch 186, fake output=8.725773811340332,real output=13.428025245666504, penalty = 4.111985206604004, 26.440977811813354
Epoch 187, fake output=8.835468292236328,real output=13.26402759552002, penalty = 4.0975751876831055, 26.910000324249268
Epoch 188, fake output=8.496658325195312,real output=13.228507041931152, penalty = 4.0391130447387695, 26.80595064163208
Epoch 189, fake output=8.2555913

Epoch 241, fake output=5.1738996505737305,real output=14.120265007019043, penalty = 3.3481693267822266, 26.815744400024414
Epoch 242, fake output=4.801599025726318,real output=14.671478271484375, penalty = 3.3494629859924316, 26.500836610794067
Epoch 243, fake output=5.3434906005859375,real output=13.758761405944824, penalty = 3.334071159362793, 26.571579933166504
Epoch 244, fake output=5.244378566741943,real output=13.601409912109375, penalty = 3.334726572036743, 26.63892650604248
Epoch 245, fake output=5.3894829750061035,real output=13.116893768310547, penalty = 3.2936692237854004, 26.397315502166748
Epoch 246, fake output=5.625978946685791,real output=13.451493263244629, penalty = 3.300097942352295, 26.696290016174316
Epoch 247, fake output=5.9549641609191895,real output=13.321022987365723, penalty = 3.287287712097168, 26.633047342300415
Epoch 248, fake output=5.658442974090576,real output=13.024312019348145, penalty = 3.2761147022247314, 26.409505605697632
Epoch 249, fake output=5.

...

Epoch 480, fake output=1.6967692375183105,real output=11.992413520812988, penalty = 1.9506839513778687, 30.60234045982361
Epoch 481, fake output=1.574318289756775,real output=11.97037124633789, penalty = 1.9371583461761475, 41.62113308906555
Epoch 482, fake output=1.5207452774047852,real output=11.588502883911133, penalty = 1.9205527305603027, 31.13352918624878
Epoch 483, fake output=1.4232631921768188,real output=12.441852569580078, penalty = 1.936124324798584, 28.951138257980347
Epoch 484, fake output=1.4203062057495117,real output=11.660074234008789, penalty = 1.920818567276001, 29.78566265106201
Epoch 485, fake output=1.64090895652771,real output=11.909017562866211, penalty = 1.935547113418579, 28.364013671875
Epoch 486, fake output=1.511279821395874,real output=12.480504035949707, penalty = 1.9274811744689941, 31.629825830459595
Epoch 487, fake output=1.4595433473587036,real output=11.757256507873535, penalty = 1.9201819896697998, 32.94292497634888
Epoch 488, fake output=1.4323072

Epoch 539, fake output=1.2538481950759888,real output=11.770537376403809, penalty = 1.7931889295578003, 29.537002086639404
Epoch 540, fake output=1.0132691860198975,real output=12.07935619354248, penalty = 1.7819938659667969, 30.20674753189087
Epoch 541, fake output=0.7129136919975281,real output=11.884092330932617, penalty = 1.7660367488861084, 28.804001808166504
Epoch 542, fake output=1.1592893600463867,real output=11.565696716308594, penalty = 1.7893427610397339, 29.716041564941406
Epoch 543, fake output=1.0507155656814575,real output=11.706469535827637, penalty = 1.7790076732635498, 28.527909994125366
Epoch 544, fake output=0.8521472215652466,real output=11.872201919555664, penalty = 1.7721272706985474, 30.014026403427124
Epoch 545, fake output=1.2759767770767212,real output=11.59393310546875, penalty = 1.767350673675537, 28.988993406295776
Epoch 546, fake output=0.9562628269195557,real output=11.867695808410645, penalty = 1.766874074935913, 27.943791389465332
Epoch 547, fake outpu

Epoch 598, fake output=0.9745030999183655,real output=11.430627822875977, penalty = 1.6751385927200317, 28.627017498016357
Epoch 599, fake output=0.8869665265083313,real output=11.663997650146484, penalty = 1.6722180843353271, 27.965993881225586
Epoch 600, fake output=0.6983336806297302,real output=11.519211769104004, penalty = 1.672320008277893, 29.42419409751892
Epoch 601, fake output=0.943359375,real output=11.18685245513916, penalty = 1.6723592281341553, 28.740575075149536
Epoch 602, fake output=1.0083377361297607,real output=11.888787269592285, penalty = 1.6830838918685913, 28.1542751789093
Epoch 603, fake output=0.81033855676651,real output=11.814282417297363, penalty = 1.6721384525299072, 27.62191915512085
Epoch 604, fake output=0.8365259766578674,real output=11.766642570495605, penalty = 1.6623780727386475, 27.610472917556763
Epoch 605, fake output=0.8950492739677429,real output=11.276826858520508, penalty = 1.6656625270843506, 27.368005752563477
Epoch 606, fake output=1.034703

Epoch 657, fake output=0.40778276324272156,real output=11.900705337524414, penalty = 1.5935308933258057, 157.7098925113678
Epoch 658, fake output=0.2842610478401184,real output=12.05639362335205, penalty = 1.6066004037857056, 154.76723647117615
Epoch 659, fake output=0.20524786412715912,real output=12.279535293579102, penalty = 1.5978994369506836, 30.9583637714386
Epoch 660, fake output=0.06360724568367004,real output=12.223471641540527, penalty = 1.5908209085464478, 28.436001300811768
Epoch 661, fake output=0.20399999618530273,real output=12.250895500183105, penalty = 1.6048352718353271, 28.23107624053955
Epoch 662, fake output=0.36858826875686646,real output=12.205755233764648, penalty = 1.5901392698287964, 32.77416372299194
Epoch 663, fake output=0.18903806805610657,real output=11.911620140075684, penalty = 1.5917348861694336, 29.68650770187378
Epoch 664, fake output=0.16876839101314545,real output=11.804725646972656, penalty = 1.5796778202056885, 29.224546670913696
Epoch 665, fake 

Epoch 716, fake output=-0.00046443939208984375,real output=11.844813346862793, penalty = 1.55535888671875, 27.113524436950684
Epoch 717, fake output=-0.17637339234352112,real output=12.382549285888672, penalty = 1.5604737997055054, 27.5169997215271
Epoch 718, fake output=-0.045675814151763916,real output=11.71027946472168, penalty = 1.5482971668243408, 27.607001066207886
Epoch 719, fake output=0.11045205593109131,real output=12.290203094482422, penalty = 1.540098786354065, 27.157333850860596
Epoch 720, fake output=-0.01069704256951809,real output=11.66220760345459, penalty = 1.5402270555496216, 27.37112545967102
Epoch 721, fake output=-0.030245618894696236,real output=11.632458686828613, penalty = 1.5433820486068726, 27.455703020095825
Epoch 722, fake output=-0.10683693736791611,real output=11.562841415405273, penalty = 1.5514951944351196, 27.004003524780273
Epoch 723, fake output=0.1782224327325821,real output=11.583732604980469, penalty = 1.5438824892044067, 27.23999810218811
Epoch 7

Display Random generated image

In [1]:
import matplotlib.pyplot as plt
noise = tf.random.normal([1,1,1, 128])
generator = tf.keras.models.load_model(os.path.join(SAVE_PATH,"face_generator.h5"))
generated_image = generator.predict(noise)
generated_image = (generated_image + 1) * 127.5
plt.imshow(np.squeeze(generated_image).astype('uint8'))

ModuleNotFoundError: No module named 'matplotlib'